# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [35]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [36]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,290.22,48,10,2.78,AU,1699833450
1,1,margaret river,-33.9500,115.0667,289.49,82,100,6.26,AU,1699833238
2,2,ciudad lazaro cardenas,17.9583,-102.2000,301.10,78,8,2.88,MX,1699833452
3,3,longyearbyen,78.2186,15.6401,269.06,86,75,6.17,SJ,1699833452
4,4,pandan,14.0453,124.1694,300.36,86,96,6.91,PH,1699833453


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [37]:
%%capture --no-display
import warnings
warnings.filterwarnings("ignore")
city_map = city_data_df.hvplot.scatter(x='Lng', y='Lat', c='Humidity', size='Humidity', 
                            color = 'City', title='City Map with Humidity Size',
                            xlabel='Longitude', ylabel='Latitude')

city_map


:Scatter   [Lng]   (Lat,City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values

conditions = (
    (city_data_df['Max Temp'] > 70) &  
    (city_data_df['Max Temp'] < 300) &
    (city_data_df['Humidity'] < 70) &  
    (city_data_df['Cloudiness'] < 30) & 
    (city_data_df['Wind Speed'] < 10)
)

ideal_weather_df = city_data_df[conditions]
ideal_weather_df = ideal_weather_df.dropna()
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,290.22,48,10,2.78,AU,1699833450
5,5,brak,27.5496,14.2714,292.92,51,0,5.07,LY,1699833453
9,9,tindouf,27.6711,-8.1474,295.40,28,0,3.09,DZ,1699833455
15,15,hamilton,39.1834,-84.5333,283.84,60,0,1.54,US,1699833458
23,23,freeport,40.6576,-73.5832,279.96,62,0,0.00,US,1699833462
...,...,...,...,...,...,...,...,...,...,...
552,552,grandview,38.8858,-94.5330,290.51,59,0,2.57,US,1699833537
553,553,sanjwal,33.7608,72.4336,288.32,61,0,1.11,PK,1699833796
569,569,edson,53.5835,-116.4356,277.23,35,0,0.51,CA,1699833658
577,577,sangola,17.4333,75.2000,295.51,66,14,2.31,IN,1699833808


### Step 3: Create a new DataFrame called `hotel_df`.

In [39]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

# Display sample data
# YOUR CODE HERE
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,48,
5,brak,LY,27.5496,14.2714,51,
9,tindouf,DZ,27.6711,-8.1474,28,
15,hamilton,US,39.1834,-84.5333,60,
23,freeport,US,40.6576,-73.5832,62,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'type': 'hotel',
    'limit': 1,      
    'radius': radius,
    'key': geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle({lng},{lat},{radius})"
    params["bias"] = f"circle({lng},{lat},{radius})"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Convert the API response to JSON format
 
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search
blackmans bay - nearest hotel: No hotel found
brak - nearest hotel: No hotel found
tindouf - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
freeport - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
fortuna - nearest hotel: No hotel found
salalah - nearest hotel: No hotel found
hammerfest - nearest hotel: No hotel found
happy valley-goose bay - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
dukhan - nearest hotel: No hotel found
zeya - nearest hotel: No hotel found
warrnambool - nearest hotel: No hotel found
metlili chaamba - nearest hotel: No hotel found
copiapo - nearest hotel: No hotel found
neuquen - nearest hotel: No hotel found
harstad - nearest hotel: No hotel found
north platte - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
franklin - nearest hotel: No hotel found
dubbo - nearest hotel: No hotel found
khawr fakkan - nearest ho

,City,Country,Lat,Lng,Humidity,Hotel Name
0,blackmans bay,AU,-43.0167,147.3167,48,No hotel found
5,brak,LY,27.5496,14.2714,51,No hotel found
9,tindouf,DZ,27.6711,-8.1474,28,No hotel found
15,hamilton,US,39.1834,-84.5333,60,No hotel found
23,freeport,US,40.6576,-73.5832,62,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [74]:
%%capture --no-display
import geopandas as gpd
import geoviews as gv
import hvplot.pandas
import matplotlib.pyplot as plt
import os
import holoviews as hv

gdf = gpd.GeoDataFrame(hotel_df, geometry=gpd.points_from_xy(hotel_df['Lng'], hotel_df['Lat']))

map_plot = gdf.hvplot.points(geo=True, size='Humidity', color='City',  
                             hover_cols=['City', 'Country', 'Hotel Name'],
                             width=800, height=600) * gv.tile_sources.OSM * gv.feature.borders()

map_plot


:Overlay
   .Points.I  :Points   [Longitude,Latitude]   (City,Humidity,Country,Hotel Name)
   .WMTS.I    :WMTS   [Longitude,Latitude]
   .Borders.I :Feature   [Longitude,Latitude]